In [13]:
import re
from bs4 import BeautifulSoup

In [8]:
from requests import request, HTTPError
from requests.compat import urljoin, urlparse, quote, unquote
from time import sleep

def download(url, params={}, headers={}, method='GET', maximum=2):
    resp = request(url=url, params=params if method=='GET' else {},
            data=params if method=='POST' else {},
            headers=headers, method=method)
    
    
    try:
        resp.raise_for_status()
    except HTTPError as e:
        if 500 <= e.response.status_code < 600 and maximum > 0:
            print('retry')
            sleep(1) # 1초?
            resp = download(url, params, headers, method, maximum-1) # inf loop
        else:
            print(e)
        
    return resp

In [11]:
# 최대한 다양한 사이트들을 크롤링해보면
# 사이트의 구조에 익숙해질 수 있음
url = 'http://www.pythonscraping.com/pages/page3.html'
resp = download(url)
dom = BeautifulSoup(resp.text, 'html.parser')

In [14]:
dom.find_all('img', attrs={'src':re.compile(r'\d\.jpg$')})

[<img src="../img/gifts/img1.jpg"/>,
 <img src="../img/gifts/img2.jpg"/>,
 <img src="../img/gifts/img3.jpg"/>,
 <img src="../img/gifts/img4.jpg"/>,
 <img src="../img/gifts/img6.jpg"/>]

In [16]:
# $111,111,111  (\d+ (숫자가 한번이상 반복되다가) [,])* (이게 0번이상 반복) \d+([.]\d+)? (소숫점 아래 2글자)
# $111
# $111,110.00
# $111.00
dom.find_all(text=re.compile(r'\$(\d+[,])*\d+(?:[.]\d+)?\b$'))

['\n$15.00\n', '\n$10,000.52\n', '\n$10,005.00\n', '\n$0.50\n', '\n$1.50\n']

In [17]:
# Google 검색 결과 예시
url = 'https://www.google.com/search'
params = {
    'q' : ['파이썬'],
    'hl': ['ko'],
    'sxsrf': ['ALeKk00P1KJuqgmTCupZG-YsxVpVk3mRGw:1625639013354'],
    'source': ['hp'],
    'ei': ['ZUjlYJa3CsWE-gSfoJqoBA'],
    'iflsig': ['AINFCbYAAAAAYOVWdT9ryM-9TZT3LtSw2ZwMuAiXXfsN'],
    'gs_lcp': ['Cgdnd3Mtd2l6EAEYATIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJ1AAWABg8ckFaAFwAHgAgAEAiAEAkgEAmAEAqgEHZ3dzLXdperABCg'],
    'sclient': ['gws-wiz']
}

header = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

params['q'] = '파이썬'

resp = download(url, params, header, 'GET')

In [51]:
resp.request.url

'https://www.google.com/search?q=%ED%8C%8C%EC%9D%B4%EC%8D%AC&hl=ko&sxsrf=ALeKk00P1KJuqgmTCupZG-YsxVpVk3mRGw%3A1625639013354&source=hp&ei=ZUjlYJa3CsWE-gSfoJqoBA&iflsig=AINFCbYAAAAAYOVWdT9ryM-9TZT3LtSw2ZwMuAiXXfsN&gs_lcp=Cgdnd3Mtd2l6EAEYATIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJ1AAWABg8ckFaAFwAHgAgAEAiAEAkgEAmAEAqgEHZ3dzLXdperABCg&sclient=gws-wiz'

In [19]:
# HTML의 class : OOP의 class와 비슷. (다중상속 가능, 여러 개일수 있음, ...)
# HTML의 id : 단 1개만 있음!

dom = BeautifulSoup(resp.text, 'lxml')

In [50]:
result = dom.find(attrs={'id':'rso'})
res_list = result.find_all(recursive=False)
for _ in res_list:
#     print(_.find_all(recursive=False, limit=2)[-1])
    print(_.find('a').find('h3').text)
    print(_.find('a').attrs['href'])
#     break
# [(_.name, _['class']) for _ in result.find_all(recursive=False)] # 검색 결과가 9개.

파이썬 - 위키백과, 우리 모두의 백과사전
https://ko.wikipedia.org/wiki/%ED%8C%8C%EC%9D%B4%EC%8D%AC
파이썬 자습서 — Python 3.9.6 문서
https://docs.python.org/ko/3/tutorial/index.html
Python - 나무위키
https://namu.wiki/w/Python
1 파이썬 시작하기 - 왕초보를 위한 Python: 쉽게 풀어 쓴 기초 ...
https://wikidocs.net/43
Python - 파이썬이란? - 코딩의 시작, TCP School
http://tcpschool.com/python/intro


AttributeError: 'NoneType' object has no attribute 'text'

In [48]:
# dom.find_all('h3', attrs={'class':'LC20lb DKV0Md'})
for _ in dom.find_all('h3', attrs={'class':'LC20lb DKV0Md'}):
    print(_.text)
    print(_.find_parent().attrs['href'])

파이썬 - 위키백과, 우리 모두의 백과사전
https://ko.wikipedia.org/wiki/%ED%8C%8C%EC%9D%B4%EC%8D%AC
파이썬 자습서 — Python 3.9.6 문서
https://docs.python.org/ko/3/tutorial/index.html
Python - 나무위키
https://namu.wiki/w/Python
1 파이썬 시작하기 - 왕초보를 위한 Python: 쉽게 풀어 쓴 기초 ...
https://wikidocs.net/43
Python - 파이썬이란? - 코딩의 시작, TCP School
http://tcpschool.com/python/intro
모두를 위한 프로그래밍 : 파이썬 강좌소개 : edwith
https://www.edwith.org/pythonforeverybody
1 장 파이썬(Python) 입문 | 파이썬 프로그래밍 기초
http://bigdata.dongguk.ac.kr/lectures/Python/_book/%ED%8C%8C%EC%9D%B4%EC%8D%AC%EC%9E%85%EB%AC%B8.html
파이썬 입문 | 프로그래머스
https://programmers.co.kr/learn/courses/2


---

In [54]:
# 네이버 검색
url = 'https://search.naver.com/search.naver'
params = {
    'where': ['nexearch'],
    'sm': ['top_hty'],
    'fbm': ['0'],
    'ie': ['utf8'],
    'query': ['통영여행']
}

resp = download(url, params, header, 'GET')

In [55]:
dom = BeautifulSoup(resp.text, 'lxml')

In [60]:
for _ in dom.find_all('a', attrs={'class':'api_txt_lines'}):
    print(_.text.strip())
    print(_.attrs['href'])

통영 여행 놀거리 아라마린서비스 요트투어
https://blog.naver.com/obh1680/222398983097
거제 통영 여행 외도 포함 내품 핫플 코스
https://blog.naver.com/lightsout/222288330327
통영 여행코스 갈만한곳 이순신공원 바다 전망
https://blog.naver.com/suk4408/222338705742
거제 통영 여행 코스 서호시장부터 동피랑벽화마을까지
https://blog.naver.com/bada3347/222360711590
통영 여행코스 이순신공원, 통영 서피랑마을까지
https://blog.naver.com/hnmh2332/222313684322
국내 1박2일 통영 여행 디피랑 서피랑 동피랑 벽화마을 이순신공원
https://blog.naver.com/hloveh01/222386234715
거제 통영 여행 코스 통영 이순신공원 등 1박2일 여행지
https://blog.naver.com/ultrabat/222362354570
통영 가볼만한곳 이순신공원 통영 여행코스
https://blog.naver.com/jyleen/222342610447
아이와 비오는 통영여행 3박 4일 코스
https://blog.naver.com/sidna/222358406013
여름에 통영여행 일정 좀 봐주세요 여름 휴가로 통영 여행 가려고 하는데요 너무 더울 것 같아 동피랑 같은 언덕으로 많이 걸어야하는 여행지는 패스하고 추천 좀 해주세요!! 3박 4일로 있을거에요,,
https://kin.naver.com/qna/detail.nhn?d1id=9&dirId=90111&docId=393379306&qb=7Ya17JiB7Jes7ZaJ&enc=utf8&section=kin.ext&rank=1&search_sort=0&spq=0
통영 여행을 계획하고 계시다면 통영 케이블카와 스카이라인 루지, 이순신공원, 욕지도를 추천드립니다. 풍경이 아름다운 곳들도 많고 맛집들도 많다고 합니다. 자세한 내용이 담겨 있는 링크도 같이

In [78]:
def can_fetch(url, path):
    resp = download(urljoin(url, '/robots.txt'))
    allow = list()
    disallow = list()
    
    
    if resp.status_code == 200:
        useragent = '*'
        for _ in resp.text.splitlines():
            pair = _.split(':')
            if len(pair) == 2:
                if pair[0].strip() == 'User-agent':
                    useragent = pair[1]
                else:
                    if pair[1].strip() == 'Allow':
                        allow.append(pair[1].strip())
                    else:
                        disallow.append(pair[1].strip())
      
    return True if path in allow else False, allow, disallow

# can_fetch(url, '/')
can_fetch('https://www.google.com', '/search')

(False,
 [],
 ['/search',
  '/search/about',
  '/search/static',
  '/search/howsearchworks',
  '/sdch',
  '/groups',
  '/index.html?',
  '/?',
  '/?hl=',
  '/?hl=*&',
  '/?hl=*&gws_rd=ssl$',
  '/?hl=*&*&gws_rd=ssl',
  '/?gws_rd=ssl$',
  '/?pt1=true$',
  '/imgres',
  '/u/',
  '/preferences',
  '/setprefs',
  '/default',
  '/m?',
  '/m/',
  '/m/finance',
  '/wml?',
  '/wml/?',
  '/wml/search?',
  '/xhtml?',
  '/xhtml/?',
  '/xhtml/search?',
  '/xml?',
  '/imode?',
  '/imode/?',
  '/imode/search?',
  '/jsky?',
  '/jsky/?',
  '/jsky/search?',
  '/pda?',
  '/pda/?',
  '/pda/search?',
  '/sprint_xhtml',
  '/sprint_wml',
  '/pqa',
  '/palm',
  '/gwt/',
  '/purchases',
  '/local?',
  '/local_url',
  '/shihui?',
  '/shihui/',
  '/products?',
  '/product_',
  '/products_',
  '/products;',
  '/print',
  '/books/',
  '/bkshp?*q=*',
  '/books?*q=*',
  '/books?*output=*',
  '/books?*pg=*',
  '/books?*jtp=*',
  '/books?*jscmd=*',
  '/books?*buy=*',
  '/books?*zoom=*',
  '/books/about',
  '/booksright

---

In [88]:
# 공공데이터 API 활용하기
# 이 데이터는 XML 방식밖에 지원 안함.
url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson'
params = {
    'serviceKey':'ibPhO0aMSaZiYiGNKqQ+GntOVhqamQs3+pNeSYjVz3OSX1RnIKShl/7jibL7YY7t3KPLvrZYk48OrEhLYFN59A==',
    'pageNo':1,
    'numOfRows':10,
    'startCreateDt':'20200310',
    'endCreateDt':'20200315'
}
resp = download(url, params=params) # header 필요없음.. API 사용하므로

# 실제 API 사용 시 사용 설명서 (빽서)를 다 읽어보고 사용해야 함.

In [89]:
resp.text

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><accDefRate>3.2396602365</accDefRate><accExamCnt>268212</accExamCnt><accExamCompCnt>251940</accExamCompCnt><careCnt>7300</careCnt><clearCnt>834</clearCnt><createDt>2020-03-15 10:01:22.22</createDt><deathCnt>75</deathCnt><decideCnt>8162</decideCnt><examCnt>16272</examCnt><resutlNegCnt>243778</resutlNegCnt><seq>74</seq><stateDt>20200315</stateDt><stateTime>00:00</stateTime><updateDt>2020-03-15 10:01:22.22</updateDt></item><item><accDefRate>3.3180003364</accDefRate><accExamCnt>261335</accExamCnt><accExamCompCnt>243701</accExamCompCnt><careCnt>7402</careCnt><clearCnt>714</clearCnt><createDt>2020-03-14 10:29:11.11</createDt><deathCnt>72</deathCnt><decideCnt>8086</decideCnt><examCnt>17634</examCnt><resutlNegCnt>235615</resutlNegCnt><seq>73</seq><stateDt>20200314</stateDt><stateTime>00:00</stateTime><updateDt>2020-03-14 10:29:11

In [90]:
# HTML Language가 아닌 XML인데도 불구하고
# 어짜피 태그로 구성된 Markup Language이므로 마찬가지로
# BeautifulSoup을 사용해서 parsing 진행하면 됨.
dom = BeautifulSoup(resp.text, 'lxml')

In [96]:
len(dom.find_all('item'))

6

In [97]:
[_.name for _ in dom.find_all('item')[0].find_all(recursive=False)]

['accdefrate',
 'accexamcnt',
 'accexamcompcnt',
 'carecnt',
 'clearcnt',
 'createdt',
 'deathcnt',
 'decidecnt',
 'examcnt',
 'resutlnegcnt',
 'seq',
 'statedt',
 'statetime',
 'updatedt']

In [99]:
for _ in dom.find_all('item')[0].find_all(recursive=False):
    print(_.name)
    print(_.text.strip())
    print()

accdefrate
3.2396602365

accexamcnt
268212

accexamcompcnt
251940

carecnt
7300

clearcnt
834

createdt
2020-03-15 10:01:22.22

deathcnt
75

decidecnt
8162

examcnt
16272

resutlnegcnt
243778

seq
74

statedt
20200315

statetime
00:00

updatedt
2020-03-15 10:01:22.22



2주차 수업 내용 : 데이터 수집하는 방법!

  정형,        반정형(문서),         비정형
DB(RDB-ORM), HTML-XML-JSON-(CSV),  텍스트-이미지
sqlite,alchemy,   beautifulsoup, request, re -> HTTP
              DOM(구조적으로 만들어짐-상하관계),            parser(html.parser, lxml-안정적,빠른, html5lib-브라우저)
     * HTML은 not well-formed 문서이므로 실제 제작된 DOM이
       다른 모습일 수 있다.
\* 불법 조심 : robots.txt, 출처 Opt-out형태로 요청 있을 시 삭제, 그것도 없으면 API 사용.

=> Cralwer(Scraping) : 빅데이터 수집기

In [ ]:
# CSSOM
# DOM을 만들 때 CSS를 사용할 수 있음. Object 모델이므로 ... 
# selector : 특정 태그, 특정 ...을 선택.

# 다른 곳에서 다 쓰일 수 있음. (JS, CSS, HTML, ...)
# HTML -> DOM Tree (오브젝드 모델)
# CSS -> CSS Parser -> CSS Object 파일
# 같은 파일이었지만, 서로 바라보는 방식이 다름. 둘을 합쳐버림.
# => DOM, CSSOM, Render Tree

# CSS의 Selector -> h1.
# CSSOM -> 약간의 구조적/관계적이 존재. DOM보다 표현력이 훨씬 더 높음.

In [100]:
url = 'http://www.pythonscraping.com/pages/page3.html'
resp = download(url)
dom = BeautifulSoup(resp.text, 'html.parser')

In [104]:
# CSSOM에서 Selector 내용
 # nth-of-type만 지원?
dom.select('td') # td를 쓰면 온갖 종류의 열을 찾게 됨.
dom.select('td, tr') # td와 tr을 모두 찾게 됨.
dom.select('tr td') # ** tr의 자손 중에 td를 찾으라는 뜻. 즉, find(recursive=True)
dom.select('tr>td') # ** tr의 자손 중에 td만을 찾으라는 뜻. find(recursive=False)

          1열    2열     3열  ....
1행<tr>
2행<tr>
3행<tr>

select('tr') = 3개
select('td') = 행*열 [1,2,3,4,5,...]
select('tr td') = (자손)행*열 [1,2,3,4,5,...]
select('tr > td') = (자식)행*열 [1,2,3,4,5,...]
select('tr > td:nth-of-type(2)') = 1행2열, 2행2열, 3행2열, ...

[<td>
 Vegetable Basket
 </td>,
 <td>
 This vegetable basket is the perfect gift for your health conscious (or overweight) friends!
 <span class="excitingNote">Now with super-colorful bell peppers!</span>
 </td>,
 <td>
 $15.00
 </td>,
 <td>
 <img src="../img/gifts/img1.jpg"/>
 </td>,
 <td>
 Russian Nesting Dolls
 </td>,
 <td>
 Hand-painted by trained monkeys, these exquisite dolls are priceless! And by "priceless," we mean "extremely expensive"! <span class="excitingNote">8 entire dolls per set! Octuple the presents!</span>
 </td>,
 <td>
 $10,000.52
 </td>,
 <td>
 <img src="../img/gifts/img2.jpg"/>
 </td>,
 <td>
 Fish Painting
 </td>,
 <td>
 If something seems fishy about this painting, it's because it's a fish! <span class="excitingNote">Also hand-painted by trained monkeys!</span>
 </td>,
 <td>
 $10,005.00
 </td>,
 <td>
 <img src="../img/gifts/img3.jpg"/>
 </td>,
 <td>
 Dead Parrot
 </td>,
 <td>
 This is an ex-parrot! <span class="excitingNote">Or maybe he's only resting?</span>
 </t

In [107]:
[_.text.strip() for _ in dom.select('tr > td:nth-of-type(3)')]

['$15.00', '$10,000.52', '$10,005.00', '$0.50', '$1.50']

In [ ]:
* 중요
tr > td:nth of type 각 행에서 자식 열의 몇 번째
     td:nth of type 모든 열에서 몇 번째
tr td:nth of type   각 행의 모든 열의 몇 번째

In [112]:
# dom.select('tr > td:nth-of-type(4) > img')
# dom.select('tr > td:nth-of-type(4) img[src$=jpg]')
dom.select('tr > td:nth-of-type(4) img[src^=..]')   # ??

SelectorSyntaxError: Malformed attribute selector at position 26
  line 1:
tr > td:nth-of-type(4) img[src^=..]
                          ^

In [116]:
len(dom.select('#gift1')), dom.select('.gift')

(1,
 [<tr class="gift" id="gift1"><td>
  Vegetable Basket
  </td><td>
  This vegetable basket is the perfect gift for your health conscious (or overweight) friends!
  <span class="excitingNote">Now with super-colorful bell peppers!</span>
  </td><td>
  $15.00
  </td><td>
  <img src="../img/gifts/img1.jpg"/>
  </td></tr>,
  <tr class="gift" id="gift2"><td>
  Russian Nesting Dolls
  </td><td>
  Hand-painted by trained monkeys, these exquisite dolls are priceless! And by "priceless," we mean "extremely expensive"! <span class="excitingNote">8 entire dolls per set! Octuple the presents!</span>
  </td><td>
  $10,000.52
  </td><td>
  <img src="../img/gifts/img2.jpg"/>
  </td></tr>,
  <tr class="gift" id="gift3"><td>
  Fish Painting
  </td><td>
  If something seems fishy about this painting, it's because it's a fish! <span class="excitingNote">Also hand-painted by trained monkeys!</span>
  </td><td>
  $10,005.00
  </td><td>
  <img src="../img/gifts/img3.jpg"/>
  </td></tr>,
  <tr class="gift"

In [ ]:
# 앞선 예시들이 이 selector을 사용하면 얼마나 간단해지는 지!

In [155]:
url = 'https://www.google.com/search'
params = {
    'q' : ['파이썬'],
    'hl': ['ko'],
    'sxsrf': ['ALeKk00P1KJuqgmTCupZG-YsxVpVk3mRGw:1625639013354'],
    'source': ['hp'],
    'ei': ['ZUjlYJa3CsWE-gSfoJqoBA'],
    'iflsig': ['AINFCbYAAAAAYOVWdT9ryM-9TZT3LtSw2ZwMuAiXXfsN'],
    'gs_lcp': ['Cgdnd3Mtd2l6EAEYATIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJzIHCCMQ6gIQJ1AAWABg8ckFaAFwAHgAgAEAiAEAkgEAmAEAqgEHZ3dzLXdperABCg'],
    'sclient': ['gws-wiz']
}

header = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

params['q'] = '파이썬'

resp = download(url, params, header, 'GET')

In [118]:
dom = BeautifulSoup(resp.text, 'html5lib')

In [121]:
len(dom.select('h3')), len(dom.select('h3.LC20lb'))\
, len(dom.select('h3.LC20lb.DKV0Md'))

(12, 8, 8)

In [122]:
# 이건 대신 find 함수와 다르게 부모로 올라갈 수 없음.
# 대신 이웃한 애들은 찾을 수 있다.
len(dom.select('br + h3'))

8

In [126]:
dom.select('br + h3')[0].find_parent()

<a data-ved="2ahUKEwjT6-W7udLxAhU_GDQIHdWEDBwQFjAAegQIBRAD" href="https://docs.python.org/ko/3/tutorial/index.html" ping="/url?sa=t&amp;source=web&amp;rct=j&amp;url=https://docs.python.org/ko/3/tutorial/index.html&amp;ved=2ahUKEwjT6-W7udLxAhU_GDQIHdWEDBwQFjAAegQIBRAD"><br/><h3 class="LC20lb DKV0Md">파이썬 자습서 — Python 3.9.6 문서</h3><div class="TbwUpd NJjxre"><cite class="iUh30 Zu0yb qLRx3b tjvcx">https://docs.python.org<span class="dyjrff qzEoUe"> › tutorial</span></cite></div></a>

In [127]:
# CSS 문법(Selector)을 사용하면 훨씬 더 간단해짐.
for _ in dom.select('br + h3'):
    print(_.text.strip())
    print(_.find_parent().attrs['href'])

파이썬 자습서 — Python 3.9.6 문서
https://docs.python.org/ko/3/tutorial/index.html
파이썬 - 위키백과, 우리 모두의 백과사전
https://ko.wikipedia.org/wiki/%ED%8C%8C%EC%9D%B4%EC%8D%AC
Python - 나무위키
https://namu.wiki/w/Python
1 파이썬 시작하기 - 왕초보를 위한 Python: 쉽게 풀어 쓴 기초 ...
https://wikidocs.net/43
Python - 파이썬이란? - 코딩의 시작, TCP School
http://tcpschool.com/python/intro
모두를 위한 프로그래밍 : 파이썬 강좌소개 : edwith
https://www.edwith.org/pythonforeverybody
1 장 파이썬(Python) 입문 | 파이썬 프로그래밍 기초
http://bigdata.dongguk.ac.kr/lectures/Python/_book/%ED%8C%8C%EC%9D%B4%EC%8D%AC%EC%9E%85%EB%AC%B8.html
파이썬을 파이썬답게 | 프로그래머스
https://programmers.co.kr/learn/courses/4008


In [128]:
# 다음 예시인 네이버..
url = 'https://search.naver.com/search.naver'
params = {
    'where': ['nexearch'],
    'sm': ['top_hty'],
    'fbm': ['0'],
    'ie': ['utf8'],
    'query': ['통영여행']
}

resp = download(url, params, header, 'GET')

In [130]:
dom = BeautifulSoup(resp.text, 'html5lib')

In [133]:
for _ in dom.select('.api_txt_lines.total_tit'):
    print(_.text.strip())
    print(_.attrs['href'])

통영 여행 놀거리 아라마린서비스 요트투어
https://blog.naver.com/obh1680/222398983097
거제 통영 여행 외도 포함 내품 핫플 코스
https://blog.naver.com/lightsout/222288330327
통영 여행코스 갈만한곳 이순신공원 바다 전망
https://blog.naver.com/suk4408/222338705742
거제 통영 여행 코스 서호시장부터 동피랑벽화마을까지
https://blog.naver.com/bada3347/222360711590
통영 여행코스 이순신공원, 통영 서피랑마을까지
https://blog.naver.com/hnmh2332/222313684322
국내 1박2일 통영 여행 디피랑 서피랑 동피랑 벽화마을 이순신공원
https://blog.naver.com/hloveh01/222386234715
거제 통영 여행 코스 통영 이순신공원 등 1박2일 여행지
https://blog.naver.com/ultrabat/222362354570
통영 가볼만한곳 이순신공원 통영 여행코스
https://blog.naver.com/jyleen/222342610447
아이와 비오는 통영여행 3박 4일 코스
https://blog.naver.com/sidna/222358406013


#### Scraping

---
### Crawler
잘 잠. 

In [188]:
from requests import request, HTTPError
from requests.compat import urljoin, urlparse, quote, unquote
from time import sleep

def download(url, params={}, headers={}, method='GET', maximum=1):
    resp = request(url=url, params=params if method=='GET' else {},
            data=params if method=='POST' else {},
            headers=headers, method=method)
    
    
    try:
        resp.raise_for_status()
    except HTTPError as e:
        if 500 <= e.response.status_code < 600 and maximum > 0:
            print('retry')
            sleep(1) # 1초?
            resp = download(url, params, headers, method, maximum-1) # inf loop
        else:
            print(e)
        
    return resp

In [189]:
def can_fetch(url, path):
    resp = download(urljoin(url, '/robots.txt'))
    allow = list()
    disallow = list()
    
    
    if resp.status_code == 200:
        useragent = '*'
        for _ in resp.text.splitlines():
            pair = _.split(':')
            if len(pair) == 2:
                if pair[0].strip() == 'User-agent':
                    useragent = pair[1]
                else:
                    if pair[1].strip() == 'Allow':
                        allow.append(pair[1].strip())
                    else:
                        disallow.append(pair[1].strip())
      
    return True if path in allow else False, allow, disallow

In [230]:
# 큐로 push, pop 구현할 거
def new_download(url):
    _ = urlparse(url)
    print(can_fetch(_[0]+'://'+_[1], _[2])[0])
    
    return download(url, headers=header)

In [191]:
Queue = list() # 주소 값 들어가있을 거
seed = 'https://www.google.com/search?q=%ED%8C%8C%EC%9D%B4%EC%8D%AC&sxsrf=ALeKk01gdzjZRape3rH9aLVuHvafXDcAhA%3A1625713882253&source=hp&ei=2mzmYMaADYnYz7sPir-w4AQ&iflsig=AINFCbYAAAAAYOZ66p1XqhEgkX96vWn92yq037Si3Vwk&oq=%ED%8C%8C%EC%9D%B4%EC%8D%AC&gs_lcp=Cgdnd3Mtd2l6EAMyBAgjECcyBAgjECcyBAgjECcyBQgAELEDMgUIABCxAzIFCAAQsQMyBQgAELEDMgUIABCxAzIFCAAQsQMyBQgAELEDOggIABCxAxCDAToCCABQsgRYyQlgtwpoAnAAeAKAAXeIAfgFkgEDMC43mAEAoAEBqgEHZ3dzLXdpeg&sclient=gws-wiz&ved=0ahUKEwjG1dKTwNLxAhUJ7HMBHYofDEwQ4dUDCAc&uact=5'

new_download(seed)

False


In [158]:
dom = BeautifulSoup(resp.text, 'lxml')

In [159]:
len(dom.select('a'))

114

In [161]:
# 링크가 항상 a href로 걸리는 것이 아니라 내부적인 JS로 돌아갈 수도 있고, 다른 경우가 있을 수 있음.
_.has_attr('href'), len(dom.select('a[href]'))

# 이게 필터링이 훨씬 더 잘 될거임.

(True, 107)

In [162]:
links = [_.attrs['href'] for _ in dom.select('a[href]')]

In [164]:
links
# 이렇게 가져와버리면 절대 주소 및 상대 주소가 뒤죽박죽 섞여버림.. 
# 상대 주소의 경우는 앞에 domain을 붙여줘야만 함.
# # : 아무 역할 안함. 

# 이런 걸 제대로 처리해줘야만 한다..

['/search?q=%ED%8C%8C%EC%9D%B4%EC%8D%AC&hl=ko&gbv=1&sei=fm3mYOCZGJX49AOviKCYAw',
 'https://support.google.com/websearch/answer/181196?hl=ko',
 'https://www.google.com/webhp?hl=ko&sa=X&ved=0ahUKEwjg0PfhwNLxAhUVPH0KHS8ECDMQPAgI',
 '#',
 '/search?q=%ED%8C%8C%EC%9D%B4%EC%8D%AC&hl=ko&source=lnms&tbm=isch&sa=X&ved=2ahUKEwjg0PfhwNLxAhUVPH0KHS8ECDMQ_AUoAXoECAEQAw',
 '/search?q=%ED%8C%8C%EC%9D%B4%EC%8D%AC&hl=ko&source=lnms&tbm=vid&sa=X&ved=2ahUKEwjg0PfhwNLxAhUVPH0KHS8ECDMQ_AUoAnoECAEQBA',
 '/search?q=%ED%8C%8C%EC%9D%B4%EC%8D%AC&hl=ko&source=lnms&tbm=bks&sa=X&ved=2ahUKEwjg0PfhwNLxAhUVPH0KHS8ECDMQ_AUoA3oECAEQBQ',
 '/search?q=%ED%8C%8C%EC%9D%B4%EC%8D%AC&hl=ko&source=lnms&tbm=nws&sa=X&ved=2ahUKEwjg0PfhwNLxAhUVPH0KHS8ECDMQ_AUoBHoECAEQBg',
 '/preferences',
 'https://docs.python.org/ko/3/tutorial/index.html',
 '#',
 'https://webcache.googleusercontent.com/search?q=cache:Rib_O5KvImcJ:https://docs.python.org/ko/3/tutorial/index.html+&cd=1&hl=ko&ct=clnk&gl=kr',
 'https://docs.python.org/ko/3/tutorial/int

In [169]:
links = [urljoin(seed, _.attrs['href']) for _ in dom.select('a[href]')
        if _.attrs['href'] != '#']

In [170]:
len(links)

94

In [251]:
directory = {'text':'TXT', 'application':'DOC', 'image':'IMG'}


def crawling(seed):
    resp = download(seed)
    #### 추가작업 -> Content-type: text/*, application/json+xml, image
    # 이런 다양한 데이터 타입때문에 이 크롤러는 죽을 것임. 수정 필요
    ctype = resp.headers['Content-type'].split('/')
    if ctype[0] in ['text', 'application', 'image']:
        if ctype[0] == 'text':
            dom = BeautifulSoup(resp.text, 'lxml')

            returnlist = list()

            for _ in dom.select('a[href], img[src], iframe[src]'):
                if _.has_attr('href'):
                    if not _.attrs['href'].startswith('#'):
                        returnlist.append(urljoin(seed, _.attrs['href']))
                else:
                    returnlist.append(urljoin(seed, _.attrs['src']))

            return returnlist

        path = directory['ctype[0]']
        path += '/'+seed.split('/')[-1]+'.'+ctype[-1]
        with open(path, 'wb') as f:
            f.write(resp.content)
                
    return None
    
#     return [urljoin(seed, _.attrs['href'])
#              for _ in dom.select('a[href]') 
#              if not _.attrs['href'].startswith('#')] #원래는 검사를 실제로 해줘야 하지만 지금은 대충.. 나중에 정규식으로 바꿔주는게 좋을 것

In [183]:
 # #gotohere : 내부주소 - 스크롤 위치를 옮기는 주소 등임. 실제 페이지와는 연관이 없음.
urlparse('https://www.google.com/search#gotohere').fragment

'gotohere'

In [254]:
seed_list = []
seed_list.append(('https://comic.naver.com/webtoon/weekdayList?week=thu', 1))
# (url, 깊이)
visited_list = []
count = 1

while seed_list:
    seed = seed_list.pop(0) # Queue, FIFO => BFS
#     seed = seed_list.pop(-1) # Stack, LIFO => DFS : INF 루프에 빠질 수 있음. - 한 페이지에 대해 깊이 찾는 방식으로는 낫배드
    url_list = crawling(seed[0])
    
    print('{} {}'.format(count, seed)) # 원래는 몇번째 사이트, 몇번째 페이지 뭐 이런 정보들이 나와야만 함.
    
    if url_list:
        for _ in url_list:
            if _ not in [link[0] for link in visited_list]\
            and _ not in [link[0] for link in seed_list]:
                seed_list.append((_, seed[1]+1))

    print('찾은 갯수: {} 방문할 곳:{} 방문한 곳:{}'.format(
        len(url_list), len(seed_list), len(visited_list)))
    
    visited_list.append(seed)
    count += 1
    
    if seed[1] > 3:
        break
    
#     if count > 100:
#         break

1 ('https://comic.naver.com/webtoon/weekdayList?week=thu', 1)
찾은 갯수: 377 방문할 곳:216 방문한 곳:0
2 ('https://www.naver.com', 2)
찾은 갯수: 289 방문할 곳:437 방문한 곳:1
3 ('https://comic.naver.com/index', 2)
찾은 갯수: 170 방문할 곳:484 방문한 곳:2
4 ('https://novel.naver.com', 2)
찾은 갯수: 199 방문할 곳:650 방문한 곳:3
5 ('https://comic.naver.com/webtoon/list.nhn?titleId=773459', 2)
찾은 갯수: 135 방문할 곳:666 방문한 곳:4
6 ('https://comic.naver.com/webtoon/weekday', 2)
찾은 갯수: 1435 방문할 곳:1419 방문한 곳:5
7 ('https://comic.naver.com/genre/bestChallenge', 2)
찾은 갯수: 223 방문할 곳:1510 방문한 곳:6
8 ('https://comic.naver.com/genre/challenge', 2)
찾은 갯수: 225 방문할 곳:1603 방문한 곳:7
9 ('https://comic.naver.com/mypage/favorite', 2)
찾은 갯수: 0 방문할 곳:1602 방문한 곳:8
10 ('https://series.naver.com/comic/recommendList.nhn', 2)
찾은 갯수: 99 방문할 곳:1684 방문한 곳:9
11 ('https://series.naver.com/novel/recommendList.nhn', 2)
찾은 갯수: 127 방문할 곳:1751 방문한 곳:10
12 ('https://comic.naver.com/webtoon/genre', 2)
찾은 갯수: 987 방문할 곳:2094 방문한 곳:11
13 ('https://comic.naver.com/webtoon/creation', 2

KeyError: 'ctype[0]'

['A']

In [138]:
urlparse('https://www.google.com/search?sfadfafd')

ParseResult(scheme='https', netloc='www.google.com', path='/search', params='', query='sfadfafd', fragment='')